## Loading Data


### Load all 39 years of ERA-Interim 3-hourly `tas` data

ERA-Interim is stored under `/g/data/ub4`

The variable for surface air temperature is `tas`, which you can find under `/g/data/ub4/erai/netcdf/3hr/atmos/oper_fc_sfc/v01/tas/tas*.nc`.

If you can't access ERA-Interim, try the ACCESS CMIP5 data under `/g/data/rr3/publications/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/3hr/atmos/3hr/r1i1p1/latest/tas/tas*.nc`

If you're working on your laptop instead of VDI, you can access data remotely over OpenDAP - `http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/atmos/Amon/r1i1p1/latest/tas/tas_Amon_ACCESS1-3_historical_r1i1p1_185001-200512.nc`

<div class="accordion" id="accordion">
  <div class="card">
    <div class="card-header" id="headingOne">
      <h5 class="mb-0">
        <button class="btn btn-link" type="button" data-toggle="collapse" data-target="#collapseOne" aria-expanded="true" aria-controls="collapseOne">
          Answer
        </button>
      </h5>
    </div>
  <div>
    <div id="collapseOne" class="collapse" aria-labelledby="headingOne" data-parent="#accordion">
      <div class="card-body">
<pre><code>
import xarray
erai_tas = xarray.open_mfdataset('/g/data/ub4/erai/netcdf/3hr/atmos/oper_fc_sfc/v01/tas/tas*.nc')
</code></pre>
      </div>
    </div>
  </div>
</div>
    
[Xarray docs: netCDF](http://xarray.pydata.org/en/stable/io.html#netcdf)

In [ ]:
erai_tas_path = '/g/data/ub4/erai/netcdf/3hr/atmos/oper_fc_sfc/v01/tas/tas*.nc'
cmip_tas_path = '/g/data/rr3/publications/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/3hr/atmos/3hr/r1i1p1/latest/tas/tas*.nc'
cmip_tas_dap  = 'http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-3/historical/mon/atmos/Amon/r1i1p1/latest/tas/tas_Amon_ACCESS1-3_historical_r1i1p1_185001-200512.nc'

! ls {erai_tas_path} | head

In [ ]:
%%time

import xarray
erai_tas = xarray.open_mfdataset(erai_tas_path)
print(erai_tas)

## Datasets


### Print the `tas` variable in the dataset - what is the variable's units?

Opening files with Xarray returns a `xarray.Dataset`. Xarray only reads the metadata to start off with, so it's fine to load giant amounts of data that you later filter down to a reasonable amount.

You can access variables, attributes and coordinates using Python attribute syntax - `object.attribute`.

<div class="accordion" id="accordion">
  <div class="card">
    <div class="card-header" id="headingOne">
      <h5 class="mb-0">
        <button class="btn btn-link" type="button" data-toggle="collapse" data-target="#collapse2" aria-expanded="true" aria-controls="collapse2">
          Answer
        </button>
      </h5>
    </div>
  <div>
    <div id="collapse2" class="collapse" aria-labelledby="headingOne" data-parent="#accordion">
      <div class="card-body">
<pre><code>
print(erai_tas.tas.units)
</code></pre>
      </div>
    </div>
  </div>
</div>

[Xarray docs: Dataset contents](http://xarray.pydata.org/en/stable/data-structures.html#dataset-contents)

### How do the time coordinate attributes differ when you enable and disable time decoding?

The time coordinate is special - Xarray converts the values in the file (units of hours since 1901-01-01) to datetime objects. You can get an array of e.g. each time's month numbers using `time.dt.month`. The same works for `year`, `day`, `hour` etc.

This behaviour can be disabled by adding a parameter `decode_times=False` when you are opening the file:
```
xarray.open_dataset('/g/data/ub4/erai/netcdf/3hr/atmos/oper_fc_sfc/v01/tas/'
                    'tas_3hrs_ERAI_historical_fc-sfc_19790101_19790131.nc',
                    decode_times=False)
```

<div class="accordion" id="accordion">
  <div class="card">
    <div class="card-header" id="headingOne">
      <h5 class="mb-0">
        <button class="btn btn-link" type="button" data-toggle="collapse" data-target="#collapse3" aria-expanded="true" aria-controls="collapse3">
          Answer
        </button>
      </h5>
    </div>
  <div>
    <div id="collapse3" class="collapse" aria-labelledby="headingOne" data-parent="#accordion">
      <div class="card-body">
<pre><code>
print(xarray.open_dataset('/g/data/ub4/erai/netcdf/3hr/atmos/oper_fc_sfc/v01/tas/'
                    'tas_3hrs_ERAI_historical_fc-sfc_19790101_19790131.nc',
                    decode_times=False).time)
                    
print(xarray.open_dataset('/g/data/ub4/erai/netcdf/3hr/atmos/oper_fc_sfc/v01/tas/'
                    'tas_3hrs_ERAI_historical_fc-sfc_19790101_19790131.nc',
                    decode_times=True).time)
</code></pre>
      </div>
    </div>
  </div>
</div>

[Xarray docs: Reading encoded data](http://xarray.pydata.org/en/stable/io.html#reading-encoded-data)

## DataArrays

If you access one of the variables in the dataset, you'll get a DataArray. Just like the dataset this has coordinates and attributes, however it also works as a numpy array - you can slice it, apply math functions etc.

At the moment, since our dataset is so large, the numpy array is hidden behind a `dask.array` object. If we slice down the array we can see its values (this is really only needed if we want to print the array).

<span style="background-color:yellow;">⚠️ Beware of loading ginormous arrays into memory with `.values`, always slice them down first ⚠️</span>

[Xarray docs: DataArray](http://xarray.pydata.org/en/stable/data-structures.html#dataarray)  
[Xarray docs: Dask](http://xarray.pydata.org/en/stable/dask.html)

In [ ]:
tas = erai_tas.tas

print(tas)
print()
print(tas[1,2,3].values)

## Subsetting data

### Create a variable `tas_nino34`, which contains `tas` in the [NINO3.4](http://www.bom.gov.au/climate/enso/indices/about.shtml) area

The `.sel()` method on a DataArray or Dataset allows you to filter the selection based on the coordinate value. The arguments to the method should be the coordinate values to use: `tas.sel(lat=30.5)`. If selecting a single value it's a good idea to add the argument `method='nearest'` to get the nearest grid point to the given value.

To select a range of values combine `.sel()` with `slice()`, e.g. `tas.sel(lon=slice(60.0, 80.0))`

`.sel()` returns a view of the original Dataset or DataArray.

<div class="accordion" id="accordion">
  <div class="card">
    <div class="card-header" id="headingOne">
      <h5 class="mb-0">
        <button class="btn btn-link" type="button" data-toggle="collapse" data-target="#collapse4" aria-expanded="true" aria-controls="collapse4">
          Answer
        </button>
      </h5>
    </div>
  <div>
    <div id="collapse4" class="collapse" aria-labelledby="headingOne" data-parent="#accordion">
      <div class="card-body">
<pre><code>
tas_nino34 = tas.sel(lat=slice(5,-5), lon=slice(-170,-120))
</code></pre>
           Note that `lat` goes from north to south, and `lon` is in degrees east
      </div>
    </div>
  </div>
</div>
    
[Xarray docs: Indexing with dimension names](http://xarray.pydata.org/en/stable/indexing.html#indexing-with-dimension-names)

### Plot a single time value of the selected area onto a map

Cartopy adds map plotting and projection tools to Pyplot. Create a cartopy-enabled axis by passing in a [projection](https://scitools.org.uk/cartopy/docs/latest/crs/projections.html) when you create the axes: `ax = plt.axes(projection=ccrs.PlateCarree())` (Plate Carree is a regular lat-lon projection)

These axes can be passed in as the `ax` parameter of plotting functions.

Cartopy adds some useful methods to the axes - try `ax.coastlines()` and `ax.stock_img()`. `ax.set_global()` will set the bounds to the whole globe.

<span style="background-color:yellow;">⚠️ This will load the data from the files, make sure you select a single time value. Use the 'stop' button in the toolbar if this takes too long ⚠️</span>

<div class="accordion" id="accordion">
  <div class="card">
    <div class="card-header" id="headingOne">
      <h5 class="mb-0">
        <button class="btn btn-link" type="button" data-toggle="collapse" data-target="#collapse5" aria-expanded="true" aria-controls="collapse5">
          Answer
        </button>
      </h5>
    </div>
  <div>
    <div id="collapse5" class="collapse" aria-labelledby="headingOne" data-parent="#accordion">
      <div class="card-body">
<pre><code>
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
ax = plt.axes(projection = ccrs.PlateCarree())
tas_nino34.isel(time=0).plot(ax = ax)
ax.coastlines()
ax.set_global()
</code></pre>
      </div>
    </div>
  </div>
</div>
    
[Xarray docs: 2D Plotting](http://xarray.pydata.org/en/stable/plotting.html#two-dimensions)  
[Cartopy docs: Using cartopy with matplotlib](https://scitools.org.uk/cartopy/docs/latest/matplotlib/intro.html)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# ???

## Calculations

### Calculate the baseline by meaning `tas_nino34` between 1961 and 1990

Just like a numpy array you can calculate aggregate statistics using `.mean()`, `.stddef()`, etc. These functions can now take a list of dimensions to run the calculation over, instead of using axis numbers.

<div class="accordion" id="accordion">
  <div class="card">
    <div class="card-header" id="headingOne">
      <h5 class="mb-0">
        <button class="btn btn-link" type="button" data-toggle="collapse" data-target="#collapse6" aria-expanded="true" aria-controls="collapse6">
          Answer
        </button>
      </h5>
    </div>
  <div>
    <div id="collapse6" class="collapse" aria-labelledby="headingOne" data-parent="#accordion">
      <div class="card-body">
<pre><code>
baseline = tas_nino34.sel(time=slice('1961','1990')).mean(['time', 'lat', 'lon'])
</code></pre>
      </div>
    </div>
  </div>
</div>
    
[Xarray docs: Aggregation](http://xarray.pydata.org/en/stable/computation.html#aggregation)

### Create a plot of the NINO3.4 index over the last decade

Xarray will automaticaly work out if it should create a one or two-dimensional plot based on the data, which is good for quick plotting.

<div class="accordion" id="accordion">
  <div class="card">
    <div class="card-header" id="headingOne">
      <h5 class="mb-0">
        <button class="btn btn-link" type="button" data-toggle="collapse" data-target="#collapse7" aria-expanded="true" aria-controls="collapse7">
          Answer
        </button>
      </h5>
    </div>
  <div>
    <div id="collapse7" class="collapse" aria-labelledby="headingOne" data-parent="#accordion">
      <div class="card-body">
<pre><code>
nino34 = tas_nino34.mean(['lat','lon']).sel(time=slice('2008', None)) - baseline
nino34.plot()
</code></pre>
      </div>
    </div>
  </div>
</div>
    
[Xarray docs: 1D plotting](http://xarray.pydata.org/en/stable/plotting.html#one-dimension)

### Save the NINO3.4 timeseries to a new NetCDF file

Create a new Dataset containing the nino34 index you calculated.

Take a look at its metadata - does it have the correct variable name and units?

Check the [CF conventions](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html#description-of-file-contents) for the file-level metadata that needs to be added to a file and add it to the dataset.

Once all the metadata looks correct, save it using `.to_netcdf()` and check the contents by running `ncdump -h` on the newly created file

<div class="accordion" id="accordion">
  <div class="card">
    <div class="card-header" id="headingOne">
      <h5 class="mb-0">
        <button class="btn btn-link" type="button" data-toggle="collapse" data-target="#collapse8" aria-expanded="true" aria-controls="collapse8">
          Answer
        </button>
      </h5>
    </div>
  <div>
    <div id="collapse8" class="collapse" aria-labelledby="headingOne" data-parent="#accordion">
      <div class="card-body">
<pre><code>
nino34_dataset = xarray.Dataset({'nino34': nino34})
nino34_dataset.attrs['title'] = "NINO3.4 index calculated from ERA-Interim"
nino34_dataset.attrs['institution'] = "ARC Centre of Excellence for Climate Extremes"
nino34_dataset.attrs['source'] = "ECMWF ERA-Interim tas"
nino34_dataset.attrs['references'] = "http://www.bom.gov.au/climate/enso/indices/about.shtml"
nino34_dataset.attrs['history'] = "2018-07-11 Calculated using Jupyter notebook"

nino34_dataset.nino34.attrs['units'] = "K"

print(nino34_dataset)
print(nino34_dataset.nino34)

nino34_dataset.to_netcdf('nino34.nc', encoding={'nino34': {'zlib': True}})

! ncdump -h nino34.nc
</code></pre>

You can run shell commands from Jupyter by starting a line with `!`
      </div>
    </div>
  </div>
</div>

[Xarray docs: Writing encoded data](http://xarray.pydata.org/en/stable/io.html#writing-encoded-data)  
[netCDF4 docs: createVariable()](http://unidata.github.io/netcdf4-python/#netCDF4.Dataset.createVariable)

### Apply the analysis to other datasets

Try running the ENSO analysis on a different CMIP5 model

Put the steps we've done so far to calculate the index into a function that takes a `tas` field as input and returns a ENSO 3.4 `DataArray`